# Cleaning INPE and Kaggle data

## Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

## Load datasets

In [5]:
mode = "local"

if mode == "local":
    kaggle_data = pd.read_csv("../../dados/Canonical/amazon.csv", sep=',', engine="python")
    inpe_data = pd.read_csv("../../dados/Canonical/Focos_2018-01-01_2018-12-31.csv", sep=',', encoding='utf-8', engine="python")
elif mode == "colab":
    kaggle_data = pd.read_csv("https://raw.githubusercontent.com/Liga-IA/Amazon-Fires-Regressions/main/dados/Canonical/amazon.csv", sep=',', encoding="iso-8859-1", engine="python")
    inpe_data = pd.read_csv("https://raw.githubusercontent.com/Liga-IA/Amazon-Fires-Regressions/main/dados/Canonical/Focos_2018-01-01_2018-12-31.csv", sep=',', encoding='utf-8', engine="python")

## Prepare Data

### Converting date and number to python numeric values

In [5]:
def convert_timestamp_to_month(value):
    timestamp = str(value)
    return int(timestamp.split('/')[1])
    
def convert_timestamp_to_year(value):
    timestamp = str(value)
    return int(timestamp.split('/')[0])

def convert_timestamp_to_day_inpe(value):
    timestamp = str(value)
    return int(timestamp.split(' ')[0].split('/')[2])

def convert_timestamp_to_hour(value):
    timestamp = str(value)
    return int(timestamp.split(' ')[1].split(":")[0])
    
def convert_timestamp_to_day_kaggle(value):
    timestamp = str(value)
    return int(timestamp.split('-')[2])   
    
def process_number(value):
    if str(value).split(".")[1] == "0":
        return int(value)
    else:
        return int(float(value) * 1000)
    
inpe_data['month'] = inpe_data['datahora'].map(convert_timestamp_to_month)
inpe_data['year'] = inpe_data['datahora'].map(convert_timestamp_to_year)
inpe_data['day'] = inpe_data['datahora'].map(convert_timestamp_to_day_inpe)
inpe_data['hour'] = inpe_data['datahora'].map(convert_timestamp_to_hour)

kaggle_data['number'] = kaggle_data['number'].map(process_number)
kaggle_data['day'] = kaggle_data['date'].map(convert_timestamp_to_day_kaggle)


### Droping useless columns

In [6]:
inpe_data = inpe_data.drop(['satelite', 'datahora', 'pais'], axis=1)

In [7]:
kaggle_data = kaggle_data.drop(['date'], axis=1)

### Encoding Months

In [8]:
month_text_to_number = {
    "Janeiro": 1,
    "Fevereiro": 2,
    "Março": 3,
    "Abril": 4,
    "Maio": 5,
    "Junho": 6,
    "Julho": 7,
    "Agosto": 8,
    "Setembro": 9,
    "Outubro": 10,
    "Novembro": 11,
    "Dezembro": 12
}

def convert_month_text_to_number(value): 
    return month_text_to_number[str(value)]

kaggle_data['month'] = kaggle_data['month'].map(convert_month_text_to_number)

### Processing States

In [9]:
state_to_number = {}
counter = 0

def encode_states(value):
    global counter
    state = str(value).lower()
    state = state.replace("á", "a")
    if state == "rio":
        state = "rio de janeiro"
    
    return state
    '''
    try:
        return state_to_number[state]
    except:
        state_to_number[state] = counter
        counter += 1
        return counter - 1
    '''
kaggle_data['state'] = kaggle_data['state'].map(encode_states)
inpe_data['estado'] = inpe_data['estado'].map(encode_states)


In [10]:
inpe_data.head(1)

,estado,municipio,bioma,diasemchuva,precipitacao,riscofogo,latitude,longitude,frp,month,year,day,hour
0,amapa,MACAPA,Amazonia,0,0.2,0.1,0.057,-51.148,14.8,1,2018,2,17


Os únicos encoding que fazem sentido para as variáveis de estado, município e bioma são aqueles que criam diferentes colunas para os valores (ONE-HOT), pois uma label encoding em um algoritmo inadequado para ele como uma regressão linear poderia levar a relações inadequadas. Como isso se refere a uma decisão de modelagem, manterei aqui os valores originais, e cabe ao modelador decidir o que fazer com as colunas para cada diferente tentativa!

Sendo assim darei como pronto os pré processamentos gerais, daqui pra fentre os pré processamentos serão específicos para cada ideia de modelagem ou análise exploratória.

## Salvando os dados pré processados

In [11]:
inpe_data.to_csv("../../dados/Preprocessed/preprocessed_inpe.csv", index=False)
kaggle_data.to_csv("../../dados/Preprocessed/preprocessed_kaggle.csv", index=False)

GG Guys!!